In [1]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import os

# Set up API key
openai.api_key = "" #openai key here

In [3]:
from neo4j import GraphDatabase

# Neo4j setup
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="gf6xb4kgeZKSS8p"

In [4]:
# Initialize Neo4j driver
neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Function to retrieve data from Neo4j
def get_neo4j_data(query, parameters=None):
    with neo4j_driver.session() as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Define your Neo4j query
neo4j_query = """
MATCH (n)-[r]->(m)
RETURN n.id AS source_id, n.type AS source_type, type(r) AS relationship, m.id AS target_id, m.type AS target_type
LIMIT 200
"""

# Retrieve data from Neo4j
neo4j_data = get_neo4j_data(neo4j_query)

neo4j_context = "\n".join([
    f"{record['source_id']} ({record['source_type']}) -[{record['relationship']}]-> {record['target_id']} ({record['target_type']})"
    for record in neo4j_data
])

In [9]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=[
        {"role": "system", "content": "You are an AI assistant with access to Neo4j graph data."},
        {"role": "user", "content": f"Using the following graph data:\n{neo4j_context}\n\nInformation on Breast cancer in Knowledge graph"}
    ],
    max_tokens=1500,  
    temperature=0.7,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in response:
    if 'content' in chunk['choices'][0]['delta']:
        print(chunk['choices'][0]['delta']['content'], end="")

Here is some information on Breast cancer from the knowledge graph data:

- Breast cancer (Medical condition) may be treated with trastuzumab (Medication), chemotherapy, surgical intervention, radiation therapy, hormone therapy, and biological therapy.
- Metastatic breast cancer (Medical condition) may be prevented with therapeutic keratoplasty and ocular surface therapy.
- Triple negative breast carcinoma (Medical condition) may be treated with chemotherapy and sacituzumab govitecan (Medication).
- Luminal B (Medical condition) may develop into triple negative breast carcinoma.
- Monoclonal antibody drug conjugate sacituzumab govitecan (Medication) may cause adverse events in elderly patients and may treat metastatic breast cancer.
- Elderly patients may be treated with sacituzumab govitecan for breast cancer.
- Breast cancer is associated with the medications trastuzumab deruxtecan, vinpocetine, inavolisib, and canakinumab, among others.
- Breast cancer is also associated with the di